# Yolo Webcam Object Detection

In [1]:
import cv2,logging,traceback
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
from yolo_utils import read_classes, read_anchors, generate_colors,draw_boxes,preprocess_video
from yad2k.models.keras_yolo import yolo_head
from yolo_algo_computation import yolo_eval
import tensorflow as tf
from keras import backend as K
from keras.models import load_model

Using TensorFlow backend.


# Initialize the session and yolo models

In [10]:
sess = K.get_session()
image_shape = (720., 1280.)  
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
yolo_model = load_model("model_data/yolo.h5")

In [11]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))
scores, boxes, classes,boxes_before_nms = yolo_eval(yolo_outputs, image_shape)

# Function to predict the frames

In [5]:
def predict(sess, image_file):
    image_data = preprocess_video(image_file, model_image_size = (608, 608))
    out_scores, out_boxes, out_classes,boxes_before_nms_out = sess.run([scores,boxes,classes,boxes_before_nms],
                                                                       feed_dict={yolo_model.input:image_data,
                                                                                  K.learning_phase(): 0})
    return out_scores, out_boxes, out_classes

# Video capture

In [29]:
fontScale= 1
lineType = 2

cap = cv2.VideoCapture(cv2.CAP_DSHOW + 1)

#height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#cap.set(cv2.CAP_PROP_FPS,5)
#writer = cv2.VideoWriter('webcam_output/webcam_detection.mp4',cv2.VideoWriter_fourcc(*'DIVX'),5,(width,height) )

cap.set(cv2.CAP_PROP_FRAME_HEIGHT,720)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,1280)
cap.set(cv2.CAP_PROP_FPS,5)
writer = cv2.VideoWriter('webcam_output/webcam_detection.mp4',cv2.VideoWriter_fourcc(*'DIVX'),5,(1280,720) )


In [30]:

try:
    while True:
        ret,frame = cap.read()
        
        if ret is False:
            print('Camera might be off')
            break
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        #Operations can be done here on the video
        if image is None:
            print('None image')
        else:
            out_scores, out_boxes, out_classes = predict(sess, image)
        colors = generate_colors(class_names)
        
        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = class_names[c]
            box = out_boxes[i]
            score = out_scores[i]

            label = '{} {:.2f}'.format(predicted_class, score)
            
            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.shape[0], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.shape[1], np.floor(right + 0.5).astype('int32'))
            
            text_origin_text = (left, top - 10)
            
            cv2.rectangle(frame,(left, top),(right, bottom),colors[c],thickness=4)
            cv2.putText(frame,label,text_origin_text,cv2.FONT_HERSHEY_SIMPLEX,fontScale,colors[c],lineType)
        
        writer.write(frame)
        
        cv2.namedWindow('frame', cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except Exception as e:
    logging.error(traceback.format_exc())
finally:
    cap.release()
    writer.release()
    cv2.destroyAllWindows()